In [ ]:
import os

from hubspot import HubSpot
import pandas as pd


ACCESS_TOKEN = os.environ["HUBSPOT_ACCESS_TOKEN"]

hubspot = HubSpot(access_token=ACCESS_TOKEN)

In [ ]:
def write_json_records(df: pd.DataFrame, file_path: str):
    df.to_json(f"data/{file_path}", orient="records", indent=2)

In [ ]:
company_props = hubspot.crm.properties.core_api.get_all(object_type="companies", archived=False)

In [ ]:
# read results into DataFrame
company_props_df = pd.json_normalize(company_props.to_dict(), "results")

# assign dtypes for columns used later
write_json_records(company_props_df, "company_props.json")

In [ ]:
companies = hubspot.crm.companies.get_all(properties=["company_type", "domain", "name"], associations=["calls", "emails", "meetings", "notes", "tasks"])
companies = [c.to_dict() for c in companies]

In [ ]:
companies_df = pd.json_normalize(companies)
write_json_records(companies_df, "companies.json")

In [ ]:
# look at the unique company types defined
companies_df["properties.company_type"].unique().tofile("data/company_types.txt", sep=os.linesep)

In [ ]:
vendor_companies_df = companies_df[companies_df["properties.company_type"] == "Vendor"]
write_json_records(vendor_companies_df, "companies_vendors.json")